<a href="https://colab.research.google.com/github/taguka/Algorithms/blob/master/atlas_baseline_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install iterative-stratification

In [0]:
!pip install fastai

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# http://pytorch.org/
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

In [0]:
!pip install pretrainedmodels

In [0]:
!pip3 install kaggle
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json



!kaggle competitions download -c human-protein-atlas-image-classification
!unzip -qq train.zip -d train | awk 'BEGIN {ORS=" "} {if(NR%500==0) print "."}'
!unzip -qq test.zip -d test | awk 'BEGIN {ORS=" "} {if(NR%500==0) print "."}'
!rm test.zip
!rm train.zip

In [0]:
!unzip -qq '/content/gdrive/My Drive/external_data.zip' -d train | awk 'BEGIN {ORS=" "} {if(NR%500==0) print "."}'

In [9]:
!ls 

adc.json  gdrive  sample_data  sample_submission.csv  test  train  train.csv


In [0]:
import os
import sys 
import json
import torch
import shutil
import numpy as np 
from torch import nn
import torch.nn.functional as F 
from torch.autograd import Variable
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms as T
from sklearn.preprocessing import MultiLabelBinarizer
from imgaug import augmenters as iaa
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import pathlib
import time 
import random 
import warnings
import torchvision
import pandas as pd 
from tqdm import tqdm 
from datetime import datetime
from torch import nn,optim
from collections import OrderedDict
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from sklearn.metrics import f1_score
import collections
from torchvision import models
from pretrainedmodels.models import bninception
from collections import OrderedDict
import cv2

In [10]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
DRIVE='/content/gdrive/My Drive/atlas'

In [0]:
class DefaultConfigs(object):
    train_data = 'train/' # where is your train data
    test_data = 'test/'   # your test data
    weights = os.path.join(DRIVE, 'checkpoints/')
    best_models = os.path.join(DRIVE, 'checkpoints','best_models/')
    submit = os.path.join(DRIVE,'submit/')
    model_name = 'bninception_bcelog'
    num_classes = 28
    img_weight = 512
    img_height = 512
    channels = 3
    lr = 0.0001
    batch_size = 8
    epochs = 50

config = DefaultConfigs()

In [0]:
def get_net():
    model = bninception(pretrained="imagenet")
    model.global_pool = nn.AdaptiveAvgPool2d(1)
    model.conv1_7x7_s2 = nn.Conv2d(config.channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    model.last_linear = nn.Sequential(
                nn.BatchNorm1d(1024),
                nn.Dropout(0.5),
                nn.Linear(1024, config.num_classes),
            )
    return model

In [0]:
def save_checkpoint(state, is_best_loss,is_best_f1,fold):
    filename = config.weights + config.model_name + os.sep +str(fold) + os.sep + "checkpoint.pth.tar"
    torch.save(state, filename)
    if is_best_loss:
        shutil.copyfile(filename,"%s/%s_fold_%s_model_best_loss.pth.tar"%(config.best_models,config.model_name,str(fold)))
    if is_best_f1:
        shutil.copyfile(filename,"%s/%s_fold_%s_model_best_f1.pth.tar"%(config.best_models,config.model_name,str(fold)))

In [0]:
def rotate(img):
    rows = img.shape[0]
    cols = img.shape[1]
    angle = np.random.choice((10, 20, 30))#, 40, 50, 60, 70, 80, 90))
    rotation_M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    img = cv2.warpAffine(img, rotation_M, (cols, rows))
    return img

def rotate_bound(image, size):
    #credits http://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    angle = np.random.randint(10,180)

    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    output = cv2.resize(cv2.warpAffine(image, M, (nW, nH)), (size, size))
    return output

def perspective(img):
    rows = img.shape[0]
    cols = img.shape[1]

    shrink_ratio1 = np.random.randint(low=85, high=110, dtype=int) / 100
    shrink_ratio2 = np.random.randint(low=85, high=110, dtype=int) / 100

    zero_point = rows - np.round(rows * shrink_ratio1, 0)
    max_point_row = np.round(rows * shrink_ratio1, 0)
    max_point_col = np.round(cols * shrink_ratio2, 0)

    src = np.float32([[zero_point, zero_point], [max_point_row-1, zero_point], [zero_point, max_point_col+1], [max_point_row-1, max_point_col+1]])
    dst = np.float32([[0, 0], [rows, 0], [0, cols], [rows, cols]])

    perspective_M = cv2.getPerspectiveTransform(src, dst)

    img = cv2.warpPerspective(img, perspective_M, (cols,rows))#, borderValue=mean_pix)
    return img

def shift(img):
    rows = img.shape[0]
    cols = img.shape[1]

    shift_ratio1 = (random.random() * 2 - 1) * np.random.randint(low=3, high=15, dtype=int)
    shift_ratio2 = (random.random() * 2 - 1) * np.random.randint(low=3, high=15, dtype=int)

    shift_M = np.float32([[1,0,shift_ratio1], [0,1,shift_ratio2]])
    img = cv2.warpAffine(img, shift_M, (cols, rows))#, borderValue=mean_pix)
    return img

In [0]:
random.seed(1979)
np.random.seed(1979)
torch.manual_seed(1979)
torch.cuda.manual_seed_all(1979)

# create dataset class
class HumanDataset(Dataset):
    def __init__(self,images_df,base_path,augument=True,mode="train"):
        if not isinstance(base_path, pathlib.Path):
            base_path = pathlib.Path(base_path)
        self.images_df = images_df.copy()
        self.augument = augument
        self.images_df.Id = self.images_df.Id.apply(lambda x:base_path / x)
        self.mlb = MultiLabelBinarizer(classes = np.arange(0,config.num_classes))
        self.mlb.fit(np.arange(0,config.num_classes))
        self.mode = mode

    def __len__(self):
        return len(self.images_df)

    def __getitem__(self,index):
        X = self.read_images(index)
        if not self.mode == "test":
            labels = np.array(list(map(int, self.images_df.iloc[index].Target.split(' '))))
            y  = np.eye(config.num_classes,dtype=np.float)[labels].sum(axis=0)
        else:
            y = str(self.images_df.iloc[index].Id.absolute())
        if self.augument:
            X = self.augumentor(X)
        #X = T.Compose([T.ToPILImage(),T.ToTensor(),T.Normalize([0.08069, 0.05258, 0.05487, 0.08282], [0.13704, 0.10145, 0.15313, 0.13814])])(X)
        X = T.Compose([T.ToPILImage(),T.ToTensor()])(X)
        return X.float(),y


    def read_images(self,index):
        row = self.images_df.iloc[index]
        filename = str(row.Id.absolute())
        #use only rgb channels
        if config.channels == 4:
            images = np.zeros(shape=(512,512,4))
        else:
            images = np.zeros(shape=(512,512,3))
        if filename.split('/')[-1].startswith("ENS"):
          images = cv2.resize(np.array(Image.open(filename+'.png')),(config.img_weight,config.img_height))
        else:
          r = np.array(Image.open(filename+"_red.png")) 
          g = np.array(Image.open(filename+"_green.png")) 
          b = np.array(Image.open(filename+"_blue.png")) 
          y = np.array(Image.open(filename+"_yellow.png")) 
          images[:,:,0] = r.astype(np.uint8) 
          images[:,:,1] = g.astype(np.uint8)
          images[:,:,2] = b.astype(np.uint8)
        if config.channels == 4:
            images[:,:,3] = y.astype(np.uint8)
        images = images.astype(np.uint8)
        #images = np.stack(images,-1) 
        if config.img_height == 512:
            return images
        else:
            return cv2.resize(images,(config.img_weight,config.img_height))
    
    def augumentor(self,image):
        rnd_flip = np.random.randint(2, dtype=int)
        rnd_rotate = np.random.randint(2, dtype=int)
        rnd_zoom = np.random.randint(2, dtype=int)
        rnd_shift = np.random.randint(2, dtype=int)

        if (rnd_flip == 1):
          rnd_flip = np.random.randint(3, dtype=int) - 1
          image = cv2.flip(image, rnd_flip)

        if (rnd_rotate == 1):
          image = rotate_bound(image, config.img_weight)

        if (rnd_zoom == 1):
          image = perspective(image)

        if (rnd_shift == 1):
          image = shift(image)
        return image


In [0]:
'''
    def augumentor(self,image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Affine(shear=(-16, 16)),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                
            ])], random_order=True)
        
        image_aug = augment_img.augment_image(image)
        return image_aug

In [0]:
class Logger(object):
    def __init__(self):
        self.terminal = sys.stdout  #stdout
        self.file = None

    def open(self, file, mode=None):
        if mode is None: mode ='w'
        self.file = open(file, mode)

    def write(self, message, is_terminal=1, is_file=1 ):
        if '\r' in message: is_file=0

        if is_terminal == 1:
            self.terminal.write(message)
            self.terminal.flush()
            #time.sleep(1)

        if is_file == 1:
            self.file.write(message)
            self.file.flush()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass

In [0]:
# evaluate meters
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [0]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25,gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, x, y):
        '''Focal loss.
        Args:
          x: (tensor) sized [N,D].
          y: (tensor) sized [N,].
        Return:
          (tensor) focal loss.
        '''
        t = Variable(y).cuda()  # [N,20]

        p = x.sigmoid()
        pt = p*t + (1-p)*(1-t)         # pt = p if t > 0 else 1-p
        w = self.alpha*t + (1-self.alpha)*(1-t)  # w = alpha if t > 0 else 1-alpha
        w = w * (1-pt).pow(self.gamma)
        return F.binary_cross_entropy_with_logits(x, t, w, size_average=False)

In [0]:
def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
       lr +=[ param_group['lr'] ]

    #assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr

def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)

    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)


    else:
        raise NotImplementedError

In [0]:

def train(train_loader,model,criterion,optimizer,epoch,valid_loss,best_results,start):
    losses = AverageMeter()
    f1 = AverageMeter()
    model.train()
      
    for i,(images,target) in enumerate(train_loader):
      images = images.cuda(non_blocking=True)
      target = torch.from_numpy(np.array(target)).float().cuda(non_blocking=True)
      # compute output
      output = model(images)
      loss = criterion(output,target)
      losses.update(loss.item(),images.size(0))

      f1_batch = f1_score(target.cpu(),output.sigmoid().cpu() > 0.2,average='macro')    
      f1.update(f1_batch,images.size(0))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      print('\r',end='',flush=True)
      message = '%s %5.2f %6.1f         |         %0.3f  %0.3f           |         %0.3f  %0.4f         |         %s  %s    | %s' % (\
                "train", i/len(train_loader), epoch,
                losses.avg, f1.avg, 
                valid_loss[0], valid_loss[1], 
                str(best_results[0])[:8],str(best_results[1])[:8],
                time_to_str((timer() - start),'min'))
      print(message , end='',flush=True)
    log.write("\n")
#    log.write(message)
#    log.write("\n")
    return [losses.avg,f1.avg]

In [0]:
# 2. evaluate function
def evaluate(val_loader,model,criterion,epoch,train_loss,best_results,start):
    # only meter loss and f1 score
    losses = AverageMeter()
    f1 = AverageMeter()
    # switch mode for evaluation
    model.cuda()
    model.eval()
    with torch.no_grad():
        for i, (images,target) in enumerate(val_loader):
            images_var = images.cuda(non_blocking=True)
            target = torch.from_numpy(np.array(target)).float().cuda(non_blocking=True)
            #image_var = Variable(images).cuda()
            #target = Variable(torch.from_numpy(np.array(target)).long()).cuda()
            output = model(images_var)
            loss = criterion(output,target)
            losses.update(loss.item(),images_var.size(0))
            f1_batch = f1_score(target.cpu(),output.sigmoid().cpu().data.numpy() > 0.2,average='macro')
            f1.update(f1_batch,images_var.size(0))
            print('\r',end='',flush=True)
            message = '%s   %5.1f %6.1f         |         %0.3f  %0.3f           |         %0.3f  %0.4f         |         %s  %s    | %s' % (\
                    "val", i , epoch,                    
                    train_loss[0], train_loss[1], 
                    losses.avg, f1.avg,
                    str(best_results[0])[:8],str(best_results[1])[:8],
                    time_to_str((timer() - start),'min'))

            print(message, end='',flush=True)
        log.write("\n")
#        log.write(message)
#        log.write("\n")
        
    return [losses.avg,f1.avg]


In [0]:
# 3. test model on public dataset and save the probability matrix
def test(test_loader,model,folds):
    sample_submission_df = pd.read_csv('sample_submission.csv')
    #3.1 confirm the model converted to cuda
    filenames,labels ,submissions= [],[],[]
    model.cuda()
    model.eval()
    submit_results = []
    for i,(input,filepath) in enumerate(tqdm(test_loader)):
        #3.2 change everything to cuda and get only basename
        filepath = [os.path.basename(x) for x in filepath]
        with torch.no_grad():
            image_var = input.cuda(non_blocking=True)
            y_pred = model(image_var)
            label = y_pred.sigmoid().cpu().data.numpy()
            #print(label > 0.5)
           
            labels.append(label > 0.2)
            filenames.append(filepath)

    for row in np.concatenate(labels):
        subrow = ' '.join(list([str(i) for i in np.nonzero(row)[0]]))
        submissions.append(subrow)
    sample_submission_df['Predicted'] = submissions
    sample_submission_df.to_csv(os.path.join(DRIVE,'submit/%s_bestloss_submission.csv'%config.model_name), index=None)

In [0]:
# 4. main function
def main():
    fold = 0
    # 4.1 mkdirs
    if not os.path.exists(config.submit):
        os.makedirs(config.submit)
    if not os.path.exists(config.weights + config.model_name + os.sep +str(fold)):
        os.makedirs(config.weights + config.model_name + os.sep +str(fold))
    if not os.path.exists(config.best_models):
        os.mkdir(config.best_models)
    if not os.path.exists(os.path.join(DRIVE, 'logs/')):
        os.mkdir(os.path.join(DRIVE, 'logs/'))
    
    # 4.2 get model
    model = get_net()
    model.cuda()

    # criterion
#    optimizer = optim.SGD(model.parameters(),lr = config.lr,momentum=0.9,weight_decay=1e-4)
    optimizer = optim.Adam(model.parameters(),lr = config.lr,weight_decay=1e-7)
    criterion = nn.BCEWithLogitsLoss().cuda()
    #criterion = FocalLoss().cuda()
    #criterion = F1Loss().cuda()
    start_epoch = 0
    best_loss = 999
    best_f1 = 0
    best_results = [np.inf,0]
    val_metrics = [np.inf,0]
    resume = False
    all_files = pd.read_csv('train.csv')
    all_files_ext=pd.read_csv(os.path.join(DRIVE,'Labels.csv'))
    all_files_ext['Target']=all_files_ext['labels'].apply(lambda x: x.replace('[','').replace(']','').replace(',',''))
    all_files_ext=all_files_ext.drop(['names','labels'],axis=1)
    all_files_ext.columns=['Id','Target']
    all_files=all_files.append(all_files_ext)

    #print(all_files)
    test_files = pd.read_csv('sample_submission.csv')  

    all_files['Target_binary']=all_files['Target'].apply(lambda x: list(np.eye(28)[list(map(int,x.split(' ')))].sum(axis=0).astype(int)) )
    lst_classes=[]
    for class_id in all_files.Target.apply(lambda x: x.split(' ')):
      lst_classes.extend(class_id)

    counter=collections.Counter(lst_classes)
    counter=counter.most_common(len(counter))
    freq_classes=[class_id[0] for class_id in counter]
    all_files['least_class']=all_files['Target'].apply(lambda x: sorted(x.split(' '),key=lambda y: freq_classes.index(y))[-1])
    lst_least_class=[]
    for class_id in all_files.least_class.apply(lambda x: map(int,x.split(' '))):
      lst_least_class.extend(class_id)
    counter_least=collections.Counter(lst_least_class)   
    class_sample_counts=[counter_least[key] for key in sorted(counter_least.keys())]
    class_weights = 1./torch.Tensor(class_sample_counts)

    msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1979)
    X=np.array([item for item in all_files.Id])
    y=np.array([item for item in all_files.Target_binary])

    for train_index, test_index in msss.split(X, y):
      train_data_list=all_files.iloc[train_index]
      val_data_list=all_files.iloc[test_index]
    # load dataset
    train_gen = HumanDataset(train_data_list,config.train_data,mode="train")
    val_gen = HumanDataset(val_data_list,config.train_data,augument=False,mode="train")
    test_gen = HumanDataset(test_files,config.test_data,augument=False,mode="test")
    
    # compute weight for all the samples in the dataset
    # samples_weights contain the probability for each example in dataset to be sampled  
    train_targets = [sample for sample in train_gen.images_df['least_class']]
    train_samples_weight = [class_weights[int(class_id)] for class_id in train_targets]
    train_sampler = torch.utils.data.sampler.WeightedRandomSampler(train_samples_weight, len(train_gen))

    val_targets = [sample for sample in val_gen.images_df['least_class']]
    val_samples_weight = [class_weights[int(class_id)] for class_id in val_targets]
    val_sampler = torch.utils.data.sampler.WeightedRandomSampler(val_samples_weight, len(val_gen))

    train_loader = DataLoader(train_gen,batch_size=config.batch_size,sampler=train_sampler,shuffle=False, pin_memory=True, drop_last=True, num_workers=4)
    val_loader = DataLoader(val_gen,batch_size=config.batch_size,sampler=val_sampler, shuffle=False,pin_memory=True,drop_last=True, num_workers=4)
    test_loader = DataLoader(test_gen,1,shuffle=False,pin_memory=True,num_workers=4)
    scheduler = lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)
    start = timer()
    ############################
    checkpoint = torch.load(os.path.join(DRIVE,"checkpoints/bninception_bcelog/0/checkpoint.pth.tar"))
    model.load_state_dict(checkpoint["state_dict"])
    start_epoch = checkpoint['epoch']
    best_loss = checkpoint['best_loss']
    optimizer.load_state_dict(checkpoint['optimizer'])
    ############################
    for epoch in range(start_epoch,config.epochs):
      scheduler.step(epoch)
      # train
      lr = get_learning_rate(optimizer)
      train_metrics = train(train_loader,model,criterion,optimizer,epoch,val_metrics,best_results,start)
      # val
      val_metrics = evaluate(val_loader,model,criterion,epoch,train_metrics,best_results,start)
      # check results 
      is_best_loss = val_metrics[0] < best_results[0]
      best_results[0] = min(val_metrics[0],best_results[0])
      is_best_f1 = val_metrics[1] > best_results[1]
      best_results[1] = max(val_metrics[1],best_results[1])   
      # save model
      save_checkpoint({
                    "epoch":epoch + 1,
                    "model_name":config.model_name,
                    "state_dict":model.state_dict(),
                    "best_loss":best_results[0],
                    "optimizer":optimizer.state_dict(),
                    "fold":fold,
                    "best_f1":best_results[1],
      },is_best_loss,is_best_f1,fold)
      # print logs
      print('\r',end='',flush=True)
      log.write('%s  %5.1f %6.1f         |         %0.3f  %0.3f           |         %0.3f  %0.4f         |         %s  %s    | %s' % (\
                "best", epoch, epoch,                    
                train_metrics[0], train_metrics[1], 
                val_metrics[0], val_metrics[1],
                str(best_results[0])[:8],str(best_results[1])[:8],
                time_to_str((timer() - start),'min'))
               )
      log.write("\n")
      time.sleep(0.01)
    
#    best_model = torch.load("%s%s_fold_%s_model_best_loss.pth.tar"%(config.best_models,config.model_name,str(fold)))
#    best_model = torch.load("checkpoints/bninception_bcelog/0/checkpoint.pth.tar")#    model.load_state_dict(best_model["state_dict"])
#    test(test_loader,model,fold)


In [0]:
    scheduler = lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)
    start = timer()
    ############################
#    checkpoint = torch.load(os.path.join(DRIVE,"checkpoints/bninception_bcelog/0/checkpoint.pth.tar"))
#    model.load_state_dict(checkpoint["state_dict"])
#    start_epoch = checkpoint['epoch']
#    best_loss = checkpoint['best_loss']
#    optimizer.load_state_dict(checkpoint['optimizer'])
    ############################
    for epoch in range(start_epoch,config.epochs):
      scheduler.step(epoch)
      # train
      lr = get_learning_rate(optimizer)
      train_metrics = train(train_loader,model,criterion,optimizer,epoch,val_metrics,best_results,start)
      # val
      val_metrics = evaluate(val_loader,model,criterion,epoch,train_metrics,best_results,start)
      # check results 
      is_best_loss = val_metrics[0] < best_results[0]
      best_results[0] = min(val_metrics[0],best_results[0])
      is_best_f1 = val_metrics[1] > best_results[1]
      best_results[1] = max(val_metrics[1],best_results[1])   
      # save model
      save_checkpoint({
                    "epoch":epoch + 1,
                    "model_name":config.model_name,
                    "state_dict":model.state_dict(),
                    "best_loss":best_results[0],
                    "optimizer":optimizer.state_dict(),
                    "fold":fold,
                    "best_f1":best_results[1],
      },is_best_loss,is_best_f1,fold)
      # print logs
      print('\r',end='',flush=True)
      log.write('%s  %5.1f %6.1f         |         %0.3f  %0.3f           |         %0.3f  %0.4f         |         %s  %s    | %s' % (\
                "best", epoch, epoch,                    
                train_metrics[0], train_metrics[1], 
                val_metrics[0], val_metrics[1],
                str(best_results[0])[:8],str(best_results[1])[:8],
                time_to_str((timer() - start),'min'))
               )
      log.write("\n")
      time.sleep(0.01)

In [0]:
if __name__ == "__main__":
  os.environ["CUDA_VISIBLE_DEVICES"] = "2"
  torch.backends.cudnn.benchmark = True
  warnings.filterwarnings('ignore')

  if not os.path.exists(os.path.join(DRIVE,'logs/')):
    os.mkdir(os.path.join(DRIVE,'logs/'))

  log = Logger()
  log.open(os.path.join(DRIVE,'logs',"%s_log_train.txt"%config.model_name),mode="a")
  log.write("\n----------------------------------------------- [START %s] %s\n\n" % (datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '-' * 51))
  log.write('                           |------------ Train -------------|----------- Valid -------------|----------Best Results---------|------------|\n')
  log.write('mode     iter     epoch    |         loss   f1_macro        |         loss   f1_macro       |         loss   f1_macro       | time       |\n')
  log.write('-------------------------------------------------------------------------------------------------------------------------------\n')
  
  main()


----------------------------------------------- [START 2018-12-30 10:52:03] ---------------------------------------------------

                           |------------ Train -------------|----------- Valid -------------|----------Best Results---------|------------|
mode     iter     epoch    |         loss   f1_macro        |         loss   f1_macro       |         loss   f1_macro       | time       |
-------------------------------------------------------------------------------------------------------------------------------


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/bn_inception-52deb4733.pth" to /root/.torch/models/bn_inception-52deb4733.pth
100%|██████████| 90583868/90583868 [00:10<00:00, 8841532.42it/s] 
